In [1]:
import subprocess
import ast
import os
import json
import sys
import ast
import glob

In [49]:
#for reading desired phrase structure
def parse_tokens_from_phrase_structure(phrase_stucture_file):
    tokens = []
    with open(phrase_stucture_file, 'r') as f:
        for l in f:
            fields = l.split('\t')
            ph_tokens = ast.literal_eval(fields[3])
            tokens.extend(ph_tokens)
    return tokens

#run espeak with mbrola dict to get phonetic trancription of word
def get_phonetic_transcription(word):
    command = 'espeak -v mb-es1 -q --pho "' + word + '"'
    cmd_tokens = command.split()
    output = subprocess.check_output(cmd_tokens)
    output_str = output.decode()
    phon_seq = [l.split('\t')[0] for l in output_str.split('\n') if l and not l.startswith('_')]
    phon_seq_str = ''.join(phon_seq)
    return phon_seq, phon_seq_str

In [50]:
phonetic_dict_old = phonetic_dict

In [185]:
#PREPARE PHONETIC LEXICON
#builds a phonetic lexicon out of the words in the structure files in given directory
#phonetic_dict = {}

mt_output_dir = '/home/alp/Documents/playground-linux/SugarDub/sub_synthesis_redo/structure'

for root, direc, files in os.walk(mt_output_dir):
    for file in files:
        if file.endswith('structure.txt'):
            print(file)
            words = parse_tokens_from_phrase_structure(os.path.join(root,file))
            for word in words:
                if not word in phonetic_dict.keys():
                    phonetic_dict[word], _ = get_phonetic_transcription(word)
            

s2_7_0156.subtitle_raw.structure.txt
s2_5_0346.subtitle.structure.txt
s3_2_0031.subtitle_raw.structure.txt
s2_7_0143.subtitle_raw.structure.txt
s3_8_0370.subtitle_raw.structure.txt
s3_6_0096.subtitle_raw.structure.txt
s3_8_0289.subtitle_raw.structure.txt
s2_8_0062.subtitle_raw.structure.txt
s2_5_0387.subtitle.structure.txt
s2_4_0314.subtitle.structure.txt


In [186]:
#write dict to json file
with open('heroes_eval_phonetic_dict_proper_wsubs.txt', 'w') as file:
     file.write(json.dumps(phonetic_dict, indent=4)) # use `json.loads` to do the reverse

In [3]:
#read dict from file
with open('heroes_eval_phonetic_dict.txt', 'r') as file:
     phonetic_dict = json.load(file)

In [4]:
def generate_pho(transcription_file, output_file):
    with open(transcription_file, 'r') as f:
        transcription = f.read()
        
    command = 'espeak -v mb-es1 -q --pho --phonout=' + output_file + ' -f ' + transcription_file
    cmd_tokens = command.split()
    output = subprocess.check_output(cmd_tokens)
    output_str = output.decode()
    return output_str

In [187]:
#CREATE DEFAULT PHO FILES
mt_output_dir = '/home/alp/Documents/playground-linux/SugarDub/sub_synthesis_redo/structure/'
default_pho_dir = '/home/alp/Documents/playground-linux/SugarDub/sub_synthesis_redo/default_pho/'

for root, direc, files in os.walk(mt_output_dir):
    for file in files:
        if file.endswith('transcript.txt'):
            print(file)
            file_id = file.split('.')[0]
            
            pho_out = os.path.join(default_pho_dir, file_id + '.default.pho')
            
            generate_pho(os.path.join(root,file), pho_out)
            

s3_8_0370.subtitle_raw.transcript.txt
s2_8_0062.subtitle_raw.transcript.txt
s3_2_0031.subtitle_raw.transcript.txt
s2_7_0156.subtitle_raw.transcript.txt
s3_6_0096.subtitle_raw.transcript.txt
s3_8_0289.subtitle_raw.transcript.txt
s2_4_0314.subtitle.transcript.txt
s2_5_0346.subtitle.transcript.txt
s2_5_0387.subtitle.transcript.txt
s2_7_0143.subtitle_raw.transcript.txt


In [ ]:
#FUNCTIONS FOR SYNTHESIS BENDING
#main
def bend_pho_to_structure(phrase_structure_file, default_pho_file, output_pho_file, verbose=False):
    phrase_structure = parse_phrase_structure(phrase_structure_file)
    default_phoneme_data = parse_pho(default_pho_file)

    phoneme_seq = get_phoneme_seq(default_phoneme_data)

    if verbose: print(phoneme_seq)

    desired_phrase_info = []
    phrase_boundaries = []
    phrase_durations = []
    phrase_pausings = []
    search_index = 0
    for phrase_tokens, start_time, end_time, pause_after in phrase_structure:

        search_beginning_index = search_index
        desired_phrase_duration = (end_time - start_time)* 1000	#desired duration
        desired_pause = int(pause_after * 1000)

        if verbose: 
            print("====================")
            print('search_beginning_index', search_beginning_index)
        for token_index, token in enumerate(phrase_tokens):
            phoneme_rep = morpheme2phoneme(token, phonetic_dict)

            word_begin = phoneme_seq.find(phoneme_rep, search_beginning_index)
            

In [197]:
#FUNCTIONS FOR SYNTHESIS BENDING
#main
def bend_pho_to_structure(phrase_structure_file, default_pho_file, output_pho_file=None, bend=True, verbose=False):
    phrase_structure = parse_phrase_structure(phrase_structure_file)
    default_phoneme_data = parse_pho(default_pho_file, skip_silence=True)

    phoneme_seq = get_phoneme_seq(default_phoneme_data)

    if verbose: print(phoneme_seq)

    desired_phrase_info = []
    phrase_boundaries = []
    phrase_durations = []
    phrase_pausings = []
    phrase_begin_index = 0
    at_default_phrase_duration = 0
    for phrase_id, (phrase_tokens, start_time, end_time, pause_after) in enumerate(phrase_structure):

        search_beginning_index = phrase_begin_index
        
        if verbose: 
            print("====================")
            print('search_beginning_index', search_beginning_index)
        for token_index, token in enumerate(phrase_tokens):
            phoneme_rep = morpheme2phoneme(token, phonetic_dict)

            #word_begin = KnuthMorrisPratt(phoneme_seq, phoneme_rep, search_beginning_index)
            word_begin = search_beginning_index

            word_end = word_begin + len(phoneme_rep)
            word_length = word_end - word_begin
            if verbose: print("%s %i - %i\t %s"%(token, word_begin, word_end, ''.join(phoneme_seq[word_begin: word_end])))
            search_beginning_index = word_end
            if word_begin == -1:
                return False

        default_phrase_duration = get_duration_of_interval(default_phoneme_data, phrase_begin_index, word_end)
        at_default_phrase_duration += default_phrase_duration
        if verbose: print('--> default phrase duration', default_phrase_duration)
        if verbose: print('--> at_default_phrase_duration', at_default_phrase_duration)

        if bend:
            desired_phrase_duration = (end_time - start_time)* 1000	#desired duration
            desired_pause = int(pause_after * 1000)
            if verbose: print('--> desired phrase duration', desired_phrase_duration)
            bend_ratio = desired_phrase_duration / default_phrase_duration
        else:
            bend_ratio = 1.0
            if phrase_id + 1 < len(phrase_structure):
                sub_interval = phrase_structure[phrase_id + 1][1] - at_default_phrase_duration/1000
                desired_pause = int(sub_interval * 1000)
            else:
                desired_pause = 0

        desired_phrase_info.append(((phrase_begin_index, word_end), bend_ratio, desired_pause))
        at_default_phrase_duration += desired_pause
        
        if verbose: 
            print("====================")
            print("interval", (phrase_begin_index, word_end))
            print("ratio", bend_ratio)
            print("pause", desired_pause)
            print("====================\n")
            
            
        phrase_begin_index = search_beginning_index
    if verbose: 
        print("--------------------========desired phrasing========--------------------")
        print(desired_phrase_info)
        

    #bend the durations of the phrases and form the desired phoneme data
    bent_phoneme_data = []
    for (search_beginning_index, end_index), bend_ratio, pause_after in desired_phrase_info:
        if verbose: print('desired_phrase_info: %i-%i, %f, %f'%(search_beginning_index, end_index, bend_ratio, pause_after))
        for phoneme_info in default_phoneme_data[search_beginning_index:end_index]:
            if verbose: 
                print('default_phoneme_info', phoneme_info)
            new_duration = int(float(phoneme_info[1]) * bend_ratio)
            new_phoneme_info = (phoneme_info[0], '%i'%new_duration, phoneme_info[2])
            if verbose: 
                print('new_phoneme_info', new_phoneme_info)
                print('...')
            bent_phoneme_data.append(new_phoneme_info)

        pause_info = ('_', pause_after, [])
        bent_phoneme_data.append(pause_info)
        if verbose: 
            print('pause_info', pause_info)
        
    #kill last pause
    bent_phoneme_data.pop()
    bent_phoneme_data.append(('_', 100, []))
    bent_phoneme_data.append(('_', 1, []))

    #write bent_phoneme_data to pho file
    if output_pho_file:
        with open(output_pho_file, 'w') as f:
            for i, phoneme_info in enumerate(bent_phoneme_data):
                f.write("%s\t%s\t%s"%(phoneme_info[0], phoneme_info[1], ' '.join(phoneme_info[2])))
                if not i == len(bent_phoneme_data):
                    f.write("\n")
    
    return True, bent_phoneme_data
   
    
#for reading desired phrase structure
def parse_phrase_structure(phrase_stucture_file):
    phrase_structure = []
    with open(phrase_stucture_file, 'r') as f:
        for l in f:
            fields = l.split('\t')
            start_time = float(fields[0])
            end_time = float(fields[1])
            pause_after = float(fields[2])
            tokens = ast.literal_eval(fields[3])
            phrase_structure.append((tokens, start_time, end_time, pause_after))
    return phrase_structure


#pho file parser for reading default phoneme timings
def parse_pho(pho_file, skip_silence=False):
    phoneme_data = []
    with open(pho_file, 'r') as f:
        for l in f:
            l_elems = l.split()
            if l_elems:
                if skip_silence and l_elems[0] == '_':
                    continue
                if len(l_elems) == 2:
                    phoneme_data.append((l_elems[0], l_elems[1], []))
                else:
                    phoneme_data.append((l_elems[0], l_elems[1], l_elems[2:]))

    return phoneme_data

def get_phoneme_seq(phoneme_data):
    phoneme_list = []
    for phon_info in phoneme_data:
        phoneme_list.append(phon_info[0])
    return phoneme_list
    #return ''.join(phoneme_list)

def morpheme2phoneme(morpheme, phonetic_dict):
    try:
        return phonetic_dict[morpheme]
    except:
        print("%s not in dict"%morpheme) #TODO
        return 0

def get_duration_of_interval(phoneme_data, start_index, end_index):
    duration = 0
    for phoneme_info in phoneme_data[start_index: end_index]:
        duration += int(phoneme_info[1])
    return duration

def KnuthMorrisPratt(text, pattern, search_from):

    '''Yields all starting positions of copies of the pattern in the text.
Calling conventions are similar to string.find, but its arguments can be
lists or iterators, not just strings, it returns all matches, not just
the first one, and it does not need the whole text in memory at once.
Whenever it yields, it will have read the text exactly up to and including
the match that caused the yield.'''

    # allow indexing into pattern and protect against change during yield
    pattern = list(pattern)

    # build table of shift amounts
    shifts = [1] * (len(pattern) + 1)
    shift = 1
    for pos in range(len(pattern)):
        while shift <= pos and pattern[pos] != pattern[pos-shift]:
            shift += shifts[pos-shift]
        shifts[pos+1] = shift

    # do the actual search
    startPos = 0
    matchLen = 0
    for c in text[search_from:]:
        while matchLen == len(pattern) or \
              matchLen >= 0 and pattern[matchLen] != c:
            startPos += shifts[matchLen]
            matchLen -= shifts[matchLen]
        matchLen += 1
        if matchLen == len(pattern):
            return startPos + search_from
    
    return -1 #not found


In [183]:
#bend sample for subtitle synthesis
sample_id = 's2_7_00'
phrase_structure_file = 'sub_synthesis/structure/' + sample_id + '.subtitle.structure.txt'
default_pho_file = 'sub_synthesis/default_pho/' + sample_id + '.default.pho'
output_pho_file = sample_id + '.subunbent.pho'
def_p = bend_pho_to_structure(phrase_structure_file, default_pho_file, output_pho_file, bend=False, verbose=True)

['a', 'd', 'a', 'm', 'e', 'r', 'a', 'e', 'l', 'b', 'i', 's', 'i', 'o', 'n', 'a', 'r', 'i', 'o', 'p', 'e', 'r', 'o', 'k', 'o', 'n', 'e', 's', 't', 'o', 's', 'p', 'o', 'd', 'e', 'r', 'e', 's', 'a', 'b', 'e', 'T', 'e', 's', 'l', 'a', 'x', 'e', 'n', 't', 'e', 's', 'e', 'b', 'e', 'a', 's', 'i', 'm', 'i', 's', 'm', 'a', 'k', 'o', 'm', 'o', 'u', 'n', 'd', 'i', 'o', 's']
search_beginning_index 0
adam 0 - 4	 adam
era 4 - 7	 era
el 7 - 9	 el
visionario. 9 - 19	 bisionario
--> default phrase duration 1103
--> at_default_phrase_duration 1103
interval (0, 19)
ratio 1.0
pause 1567

search_beginning_index 19
pero 19 - 23	 pero
con 23 - 26	 kon
estos 26 - 31	 estos
poderes 31 - 38	 poderes
a 38 - 39	 a
veces 39 - 44	 beTes
la 44 - 46	 la
gente 46 - 51	 xente
se 51 - 53	 se
vea 53 - 56	 bea
si 56 - 58	 si
misma 58 - 63	 misma
como 63 - 67	 komo
un 67 - 69	 un
dios 69 - 73	 dios
--> default phrase duration 3562
--> at_default_phrase_duration 6232
interval (19, 73)
ratio 1.0
pause 0

--------------------

In [196]:
#bend sample for synthesis
sample_id = 's2_7_0247'
phrase_structure_file = 'mt_output/' + sample_id + '.translation.structure.txt'
default_pho_file = 'default_pho/' + sample_id + '.default.pho'
#output_pho_file = sample_id + '.subunbent.pho'
def_p = bend_pho_to_structure(phrase_structure_file, default_pho_file, output_pho_file=None, bend=True, verbose=True)

['k', 'e', 'a', 'T', 'e', 's', 'a', 'k', 'i', 'd', 'o', 'n', 'd', 'e', 'e', 's', 't', 'a', 'm', 'o', 's']
search_beginning_index 0
qué 0 - 2	 ke
--> default phrase duration 160
--> at_default_phrase_duration 160
--> desired phrase duration 320.0
interval (0, 2)
ratio 2.0
pause 990

search_beginning_index 2
haces 2 - 6	 aTes
aquí 6 - 9	 aki
--> default phrase duration 582
--> at_default_phrase_duration 1732
--> desired phrase duration 530.0
interval (2, 9)
ratio 0.9106529209621993
pause 600

search_beginning_index 9
dónde 9 - 14	 donde
estamos 14 - 21	 estamos
--> default phrase duration 785
--> at_default_phrase_duration 3117
--> desired phrase duration 510.0000000000002
interval (9, 21)
ratio 0.6496815286624207
pause 680

--------------------========desired phrasing========--------------------
[((0, 2), 2.0, 990), ((2, 9), 0.9106529209621993, 600), ((9, 21), 0.6496815286624207, 680)]
desired_phrase_info: 0-2, 2.000000, 990.000000
default_phoneme_info ('k', '112', [])
new_phoneme_info 

In [48]:
print(phoneme_seq)
phrase_tokens, start_time, end_time, pause_after = phrase_structure[0]
print(phrase_tokens)
phoneme_rep = morpheme2phoneme(phrase_tokens[2], phonetic_dict)
print(phoneme_rep)
word_begin = phoneme_seq.find(phoneme_rep, 0)
print(word_begin)

asetSomutSopornosotrosemosetSomutSoportitantokenopodiamosaTerparati__sitebasaabandonar____
['has', 'hecho', 'mucho', 'por', 'nosotros']
mutSo
6


In [198]:
#BEND ALL
mt_output_dir = '/home/alp/Documents/playground-linux/SugarDub/mt_output'
default_pho_dir = '/home/alp/Documents/playground-linux/SugarDub/default_pho/'
bent_pho_dir = '/home/alp/Documents/playground-linux/SugarDub/sub_synthesis_redo/unbent_pho/'

for root, direc, files in os.walk(mt_output_dir):
    for file in files:
        if file.endswith('structure.txt'):
            
            file_id = file.split('.')[0]
            print(file_id)
            
            phrase_structure_file = os.path.join(mt_output_dir, file)
            default_pho_file = os.path.join(default_pho_dir, file_id + '.default.pho')
            bent_pho_out = os.path.join(bent_pho_dir, file_id + '.unbent.redo.pho')
                                            
            all_good, ratios = bend_pho_to_structure(phrase_structure_file, default_pho_file, bend=False, verbose=False)   
            #all_good, _ = bend_pho_to_structure(phrase_structure_file, default_pho_file, bent_pho_out, bend=False, verbose=False)         
            if not all_good:
                print('problem', file_id)
                
            break


s2_10_0080


In [112]:
#Functions for synthesis (doesn't work)
def synthesize_pho(pho_file, output_wav):
    command = 'mbrola ./es1/es1 ' + pho_file + ' ' + output_wav
    cmd_tokens = command.split()
    output = subprocess.check_output(cmd_tokens)
    return output.decode()
    print(command)

In [ ]:
#SYNTHESIZE ALL BENT FILES
#~BASH: for i in `ls bent_pho`; do mbrola ./es1/es1 bent_pho/$i bent_synth/$i.wav; done